In [ ]:
!git clone https://github.com/andssuu/redes_neurais_pos.git 

Cloning into 'redes_neurais_pos'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 83 (delta 15), reused 22 (delta 2), pack-reused 0
Unpacking objects: 100% (83/83), done.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation. 

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [ ]:
import random
import numpy as np

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [ ]:
# Função de Ativação Sigmóide
def sigmoid(net):
    return 1.0/(1.0+np.exp(-net))

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

# Classe Network
class Network(object):

    def __init__(self, sizes, activation_function=sigmoid,
                 prime_function=sigmoid_prime):
        self.num_layers = len(sizes)  #número de neurônios em cada camada
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #limiar
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #pesos
        self.activation_function = activation_function
        self.prime_function = prime_function

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = self.activation_function(np.dot(w, x)+b) #net = (∑xw+b)
        return x

    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""

        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)
        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂)
            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j,acc,n_test,(acc*100)/n_test));

            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, 𝜂):
        """Atualiza os pesos e limiares da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""

        #inicializa matriz com derivadas de pesos e limiares
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            #resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
            #soma os deltas do minibatch
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        #atualiza pesos e limiares (𝜂*𝛿*f’(net)*𝑥)  
        self.weights = [w-(𝜂/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(𝜂/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # Feedforward
        activation = x

        # Lista para armazenar todas as saídas dos neurônios (z), camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores net, camada por camada
        nets = [] 

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation)+b   
            nets.append(net)
            activation = self.activation_function(net) #z = valor de saída do neurônio
            activations.append(activation)
        
        # Backward pass 
        
        #última camada -(u-z)f'(net)
        delta = self.cost_derivative(activations[-1], y) * self.prime_function(nets[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #(𝑦−𝑧)*f’(net)*𝑥

        # l = 1 significa a última camada de neurônios, l = 2 é a penúltima e assim por diante. 
        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = self.prime_function(net)
            #delta da camada intermediaria. Note que utiliza o delta calculado anteriormente
            delta = np.dot(self.weights[-l+1].transpose(), delta) * zs 
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #∑(𝛿𝑤)f’(net)𝑥
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural 
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [ ]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [ ]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

#arquitetura da rede
arquitecture = [784, 30, 20, 10]
mlp = Network(arquitecture)
mlp.SGD(training_data, 10, 32, 0.3, test_data=test_data)


Epoch 0 : 4051 / 10000 = 40.51%
Epoch 1 : 6660 / 10000 = 66.6%
Epoch 2 : 7440 / 10000 = 74.4%
Epoch 3 : 7858 / 10000 = 78.58%
Epoch 4 : 8126 / 10000 = 81.26%
Epoch 5 : 8306 / 10000 = 83.06%
Epoch 6 : 8431 / 10000 = 84.31%
Epoch 7 : 8543 / 10000 = 85.43%
Epoch 8 : 8607 / 10000 = 86.07%
Epoch 9 : 8673 / 10000 = 86.73%


##**Mini-Projeto**
1) Realizar avaliações modificando os seguintes parâmetros:     

     a) Taxa de aprendizagem: 0.1, 0.3 e 0.5
     b) Função de ativação RELU
     c) Rede com uma camada intermediária com 3 configurações diferentes (Explicite a configuração utilizada)
     d) Rede com duas camadas intermediárias com 3 configurações diferentes (Explicite a configuração utilizada)
     
      
Complete a Tabela abaixo com os resultados (Pra cada configuração de camadas intermediárias, execute as 3 taxas de aprendizagem pra função de ativação Sigmoide e Relu):


\begin{array}{|c|ccc|ccc|ccc|}\hline\\ \\
  1\;Camada\;Intermediária & & \mathcal{𝜂=0.1} & &  & \mathcal{𝜂=0.3} & & & \mathcal{𝜂=0.5} &  & \\ \hline 
Configurações & 1 & 2 & 3 & 1 & 2 & 3 & 1 & 2 & 3 & \\ \hline
Sigmoide  & 77.1 & 64.36 & 53.41 & 76.44 & 88.02 & 64.22 & 89.18 & 82.53 & 64.64 & \\ \hline 
Relu  & 9.8 &  9.8 & 9.8 & 9.8 & 9.8 & 9.8 & 9.8 & 9.8 & 9.8 & \\ \hline
  2\;Camadas\;Intermediárias & & \mathcal{𝜂=0.1} & &  & \mathcal{𝜂=0.3} & & & \mathcal{𝜂=0.5} &  & \\ \hline 
Configurações & 1 & 2 & 3 & 1 & 2 & 3 & 1 & 2 & 3 & \\ \hline 
Sigmoide  & 79.36 & 68.06 & 76.38 & 78.98 & 89.05 & 80.34 & 90.12 & 90.49 & 91.46 & \\ \hline
Relu  & 9.8 & 9.8 & 9.8 & 9.8 & 9.8 & 9.8 & 9.8 & 9.8 & 9.8 & \\ \hline
\end{array}

2) Modifique a taxa de aprendizagem pra diminuir com o tempo. Execute com a melhor configuração encontrada. Melhorou o resultado?


**Resolução Questão 1**

Definição da função Relu e sua derivada

In [ ]:
def relu(net):
    return np.maximum(0, net)

def relu_prime(z):
    return (z > 0).astype(float)

Execução da rede neural com **1 camada intermediária**


As variações das configurações serão em relação ao número de neurônios na camada intermediária.

Os demais parâmetros tais como: **número de épocas** e **tamanho do batch** permanecem constantes.

In [ ]:
learning_rates = [0.1, 0.3, 0.5]
n_neurons = [28, 56, 84]
functions = [[sigmoid, sigmoid_prime], [relu, relu_prime]]
for learning_rate in learning_rates:
    for _n, conf in enumerate(n_neurons):
        for _function in functions:
            training_data, validation_data, test_data = load_data_wrapper()
            print('Taxa de aprendizagem (𝜂): {}, Configuração {} ('
                    'Nº neurônios: {}), Função: {}'.format(learning_rate, _n+1, conf,
                                                            _function[0].__name__))
            arquitecture = [784, conf, 10]
            mlp = Network(arquitecture, activation_function=_function[0],
                            prime_function=_function[1])
            mlp.SGD(training_data, 10, 32,
                    learning_rate, test_data=test_data)

Taxa de aprendizagem (𝜂): 0.1, Configuração 1 (Nº neurônios: 28), Função: sigmoid
Epoch 0 : 3098 / 10000 = 30.98%
Epoch 1 : 3893 / 10000 = 38.93%
Epoch 2 : 4997 / 10000 = 49.97%
Epoch 3 : 5586 / 10000 = 55.86%
Epoch 4 : 6075 / 10000 = 60.75%
Epoch 5 : 6470 / 10000 = 64.7%
Epoch 6 : 6812 / 10000 = 68.12%
Epoch 7 : 7125 / 10000 = 71.25%
Epoch 8 : 7397 / 10000 = 73.97%
Epoch 9 : 7710 / 10000 = 77.1%
Taxa de aprendizagem (𝜂): 0.1, Configuração 1 (Nº neurônios: 28), Função: relu
Epoch 0 : 980 / 10000 = 9.8%
Epoch 1 : 980 / 10000 = 9.8%
Epoch 2 : 980 / 10000 = 9.8%
Epoch 3 : 980 / 10000 = 9.8%
Epoch 4 : 980 / 10000 = 9.8%
Epoch 5 : 980 / 10000 = 9.8%
Epoch 6 : 980 / 10000 = 9.8%
Epoch 7 : 980 / 10000 = 9.8%
Epoch 8 : 980 / 10000 = 9.8%
Epoch 9 : 980 / 10000 = 9.8%
Taxa de aprendizagem (𝜂): 0.1, Configuração 2 (Nº neurônios: 56), Função: sigmoid
Epoch 0 : 2764 / 10000 = 27.64%
Epoch 1 : 3876 / 10000 = 38.76%
Epoch 2 : 4604 / 10000 = 46.04%
Epoch 3 : 5118 / 10000 = 51.18%
Epoch 4 : 5519 / 1000

Execução da rede neural com **2 camadas intermediárias**

In [ ]:
#arquitetura da rede com 2 camada intermediária
learning_rates = [0.1, 0.3, 0.5]
n_neurons = [[28, 28], [56, 56], [84, 84]]
functions = [[sigmoid, sigmoid_prime], [relu, relu_prime]]
for learning_rate in learning_rates:
    for _n, conf in enumerate(n_neurons):
        for _function in functions:
            training_data, validation_data, test_data = load_data_wrapper()
            print('Taxa de aprendizagem (𝜂): {}, Configuração {} ('
                'Nº neurônios: {}), Função: {}'.format(learning_rate, _n+1, conf,
                                                        _function[0].__name__))
            arquitecture = [784, conf[0], conf[1], 10]
            mlp = Network(arquitecture, activation_function=_function[0],
                        prime_function=_function[1])
            mlp.SGD(list(training_data), 10, 32,
                    learning_rate, test_data=test_data)


Taxa de aprendizagem (𝜂): 0.1, Configuração 1 (Nº neurônios: [28, 28]), Função: sigmoid
Epoch 0 : 2826 / 10000 = 28.26%
Epoch 1 : 4330 / 10000 = 43.3%
Epoch 2 : 5383 / 10000 = 53.83%
Epoch 3 : 6198 / 10000 = 61.98%
Epoch 4 : 6738 / 10000 = 67.38%
Epoch 5 : 7087 / 10000 = 70.87%
Epoch 6 : 7378 / 10000 = 73.78%
Epoch 7 : 7641 / 10000 = 76.41%
Epoch 8 : 7803 / 10000 = 78.03%
Epoch 9 : 7936 / 10000 = 79.36%
Taxa de aprendizagem (𝜂): 0.1, Configuração 1 (Nº neurônios: [28, 28]), Função: relu
Epoch 0 : 980 / 10000 = 9.8%
Epoch 1 : 980 / 10000 = 9.8%
Epoch 2 : 980 / 10000 = 9.8%
Epoch 3 : 980 / 10000 = 9.8%
Epoch 4 : 980 / 10000 = 9.8%
Epoch 5 : 980 / 10000 = 9.8%
Epoch 6 : 980 / 10000 = 9.8%
Epoch 7 : 980 / 10000 = 9.8%
Epoch 8 : 980 / 10000 = 9.8%
Epoch 9 : 980 / 10000 = 9.8%
Taxa de aprendizagem (𝜂): 0.1, Configuração 2 (Nº neurônios: [56, 56]), Função: sigmoid
Epoch 0 : 2996 / 10000 = 29.96%
Epoch 1 : 4830 / 10000 = 48.3%
Epoch 2 : 5470 / 10000 = 54.7%
Epoch 3 : 5917 / 10000 = 59.17%
Epoc

**Resolução Questão 2**

De acordo com a tabela, a melhor configuração foi a rede MLP com a seguinte configuração:

*   2 camadas intermediárias
*   Taxa de aprendizagem 0.5
*   Quantidade de 84 neurônios em cada camada intermediária
*   Acurácia de 91.46%

A estratégia do código a seguir é alterar a taxa de aprendizagem baseado no número épocas


In [ ]:
# Classe Network
class NetworkChangeRate(Network):

    def __init__(self, sizes, activation_function=sigmoid,
                 prime_function=sigmoid_prime):
        super().__init__(sizes, activation_function=sigmoid,
                 prime_function=sigmoid_prime)

    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""
        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)
        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
        for j in range(epochs):
            if ((j+1)/epochs > 0.8):
                _n = 0.425
            elif ((j+1)/epochs > 0.5):
                _n = 0.45
            else:
                _n = 0.5
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂)
            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j,acc,n_test,(acc*100)/n_test));
            else:
                print("Epoch {} finalizada".format(j))

learning_rates = [0.5]
n_neurons = [[84, 84]]
functions = [[sigmoid, sigmoid_prime]]
for learning_rate in learning_rates:
    for _n, conf in enumerate(n_neurons):
        for _function in functions:
            training_data, validation_data, test_data = load_data_wrapper()
            print('Taxa de aprendizagem (𝜂): {},'\
                'Nº neurônios: {}, Função: {}'.format(learning_rate, conf,
                                                        _function[0].__name__))
            arquitecture = [784, conf[0], conf[1], 10]
            mlp = NetworkChangeRate(arquitecture, activation_function=_function[0],
                        prime_function=_function[1])
            mlp.SGD(list(training_data), 10, 32,
                    learning_rate, test_data=test_data)


Taxa de aprendizagem (𝜂): 0.5,Nº neurônios: [84, 84], Função: sigmoid
Epoch 0 : 6287 / 10000 = 62.87%
Epoch 1 : 6951 / 10000 = 69.51%
Epoch 2 : 7808 / 10000 = 78.08%
Epoch 3 : 8570 / 10000 = 85.7%
Epoch 4 : 8786 / 10000 = 87.86%
Epoch 5 : 8893 / 10000 = 88.93%
Epoch 6 : 8979 / 10000 = 89.79%
Epoch 7 : 9022 / 10000 = 90.22%
Epoch 8 : 9074 / 10000 = 90.74%
Epoch 9 : 9114 / 10000 = 91.14%


A estratégia utilizada, na mudança da taxa de aprendizagem, não resultou em melhoria da acurácia. Entretanto, é importante destacar que a quantidade de épocas foi pequena e consequentemente dificultou o efeito dessa estratégia
